In [13]:
from src.utils.database import get_async_engine, get_async_session, get_async_engine_session
from src.sechema.inference import Video, Frame, Inference

In [ ]:
from sqlalchemy import create_engine
import asyncio
from sqlalchemy.ext.asyncio import create_async_engine, AsyncSession
from sqlalchemy.orm import sessionmaker, relationship, declarative_base
from sqlalchemy import Column, Integer, String, Float, ForeignKey, TIMESTAMP, func, ARRAY, text

In [6]:
user = "postgres"
password = "postgrespassword"
host = "localhost"
port = "5432"
database = "ai_db"
engine, AsyncSessionLocal = get_async_engine_session(user, password, host, port, database)

In [8]:
schema_name = "video_inference"
Base = declarative_base()

# Ensure schema exists
async with engine.begin() as conn:
    await conn.execute(text(f"CREATE SCHEMA IF NOT EXISTS {schema_name}"))
    # Create all tables
    await conn.run_sync(Base.metadata.create_all)

2026-02-18 16:59:25,149 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2026-02-18 16:59:25,150 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 16:59:25,156 INFO sqlalchemy.engine.Engine select current_schema()
2026-02-18 16:59:25,157 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 16:59:25,160 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2026-02-18 16:59:25,160 INFO sqlalchemy.engine.Engine [raw sql] ()
2026-02-18 16:59:25,163 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 16:59:25,164 INFO sqlalchemy.engine.Engine CREATE SCHEMA IF NOT EXISTS video_inference
2026-02-18 16:59:25,164 INFO sqlalchemy.engine.Engine [generated in 0.00046s] ()
2026-02-18 16:59:25,167 INFO sqlalchemy.engine.Engine COMMIT


In [9]:
# Insert example data
async with AsyncSessionLocal() as session:
    video = Video(file_path="../Data/Videos/leak.mp4")
    frame = Frame(frame_number=1, frame_path="frames/sample_frame_1.jpg")
    video.frames.append(frame)

    session.add(video)
    await session.commit()       # must await in async
    await session.refresh(video) # refresh to get ID
    await session.refresh(frame) # optional, get frame ID

    print("Video saved with ID:", video.id)
    print("Frame saved with ID:", frame.id)

2026-02-18 16:59:28,671 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 16:59:28,673 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.videos (file_path) VALUES ($1::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at
2026-02-18 16:59:28,674 INFO sqlalchemy.engine.Engine [generated in 0.00104s] ('../Data/Videos/leak.mp4',)
2026-02-18 16:59:28,683 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.frames (video_id, frame_number, frame_path) VALUES ($1::INTEGER, $2::INTEGER, $3::VARCHAR) RETURNING video_inference.frames.id, video_inference.frames.created_at
2026-02-18 16:59:28,684 INFO sqlalchemy.engine.Engine [generated in 0.00089s] (2, 1, 'frames/sample_frame_1.jpg')
2026-02-18 16:59:28,690 INFO sqlalchemy.engine.Engine COMMIT
2026-02-18 16:59:28,711 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 16:59:28,713 INFO sqlalchemy.engine.Engine SELECT video_inference.videos.id, video_inference.videos.file_path, video_inference.v

#### Insert the videos first and them frames

In [12]:
# Insert video first
async with AsyncSessionLocal() as session:
    # 1. Create Video object
    video = Video(file_path="../Data/Videos/corrosion.mp4")
    
    # 2. Add and commit video to get its ID
    session.add(video)
    await session.commit()
    await session.refresh(video)  # get video.id from DB

    print("Video saved with ID:", video.id)

    # 3. Create Frame object and link to the existing video
    frame = Frame(frame_number=1, frame_path="frames/sample_frame_1.jpg", video_id=video.id)
    
    # Alternatively, you can still use the relationship:
    # video.frames.append(frame)

    # 4. Add frame to session and commit
    session.add(frame)
    await session.commit()
    await session.refresh(frame)

    print("Frame saved with ID:", frame.id)

2026-02-18 17:04:28,897 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 17:04:28,898 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.videos (file_path) VALUES ($1::VARCHAR) RETURNING video_inference.videos.id, video_inference.videos.created_at
2026-02-18 17:04:28,898 INFO sqlalchemy.engine.Engine [cached since 300.2s ago] ('../Data/Videos/corrosion.mp4',)
2026-02-18 17:04:28,903 INFO sqlalchemy.engine.Engine COMMIT
2026-02-18 17:04:28,907 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2026-02-18 17:04:28,908 INFO sqlalchemy.engine.Engine SELECT video_inference.videos.id, video_inference.videos.file_path, video_inference.videos.created_at 
FROM video_inference.videos 
WHERE video_inference.videos.id = $1::INTEGER
2026-02-18 17:04:28,909 INFO sqlalchemy.engine.Engine [cached since 300.2s ago] (3,)
Video saved with ID: 3
2026-02-18 17:04:28,914 INFO sqlalchemy.engine.Engine INSERT INTO video_inference.frames (video_id, frame_number, frame_path) VALUES ($1::INTEGER, $2: